In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import mylib.data_loaders as data_loaders
import mylib.data_transformers as data_transformers
import mylib.datasets as ds
import mylib.models_repo as models_repo
import mylib.optimizer_repo as optimizer_repo
import mylib.scheduler_repo as scheduler_repo
import mylib.trainer as trainer
import mylib.model_saver as model_saver

In [3]:
from torchvision.datasets.folder import ImageFolder

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [4]:
%matplotlib inline

In [5]:
data_path = "/home/as/datasets/kaggle.dog.breed/train"
csv_file = "/home/as/datasets/kaggle.dog.breed/labels.csv"
sample_submission_file = '/home/as/datasets/kaggle.dog.breed/sample_submission.csv'

num_classes = 120
img_size  = 224
scale_img_size = 300
batch_size = 128

In [6]:
norm = data_transformers.pytorch_zoo_normaliser
trans = data_transformers.get_transformer(img_size, scale_img_size, norm, False)
trans_aug = data_transformers.get_transformer(img_size, scale_img_size, norm, True)
trans_valid = data_transformers.get_test_valid_transformer(img_size, scale_img_size, norm)

In [7]:
#np.random.seed(42)
labels = pd.read_csv(csv_file)
mask = np.random.rand(len(labels)) < 0.75
labels_train = labels[mask].values
labels_valid = labels[~mask].values
sub = pd.read_csv(sample_submission_file,)
breeds = list(sub)[1:]

train_images = ds.DatasetFromCSV(labels_train, breeds, data_path, 'jpg', transform=trans)
train_images_aug = ds.DatasetFromCSV(labels_train, breeds, data_path, 'jpg', transform=trans_aug)
valid_images = ds.DatasetFromCSV(labels_valid, breeds, data_path, 'jpg', transform=trans_valid)

In [8]:
len(breeds)

120

In [9]:
print('Number of train instances', len(train_images))
print('Number of valid instances', len(valid_images))

Number of train instances 7652
Number of valid instances 2570


In [10]:
print('Classes', train_images.classes)

Classes ['affenpinscher', 'afghan_hound', 'african_hunting_dog', 'airedale', 'american_staffordshire_terrier', 'appenzeller', 'australian_terrier', 'basenji', 'basset', 'beagle', 'bedlington_terrier', 'bernese_mountain_dog', 'black-and-tan_coonhound', 'blenheim_spaniel', 'bloodhound', 'bluetick', 'border_collie', 'border_terrier', 'borzoi', 'boston_bull', 'bouvier_des_flandres', 'boxer', 'brabancon_griffon', 'briard', 'brittany_spaniel', 'bull_mastiff', 'cairn', 'cardigan', 'chesapeake_bay_retriever', 'chihuahua', 'chow', 'clumber', 'cocker_spaniel', 'collie', 'curly-coated_retriever', 'dandie_dinmont', 'dhole', 'dingo', 'doberman', 'english_foxhound', 'english_setter', 'english_springer', 'entlebucher', 'eskimo_dog', 'flat-coated_retriever', 'french_bulldog', 'german_shepherd', 'german_short-haired_pointer', 'giant_schnauzer', 'golden_retriever', 'gordon_setter', 'great_dane', 'great_pyrenees', 'greater_swiss_mountain_dog', 'groenendael', 'ibizan_hound', 'irish_setter', 'irish_terrier

In [11]:
train_loader = data_loaders.get_data_loader(train_images, batch_size)
train_loader_aug = data_loaders.get_data_loader(train_images_aug, batch_size)
valid_loader = data_loaders.get_data_loader(valid_images, batch_size)

In [12]:
if False:
    model = models_repo.resnet_vanilla(num_classes)
    criteria, optimizer = optimizer_repo.sgd(model, 0.01)
    scheduler = scheduler_repo.step_lr(optimizer)

    best_model = trainer.experiment(model, criteria, optimizer, scheduler, train_loader, valid_loader, 10)

In [13]:
if False:
    model = models_repo.resnet_vanilla(num_classes)
    criteria, optimizer = optimizer_repo.sgd(model, 0.01)
    scheduler = scheduler_repo.step_lr(optimizer)

    best_model = trainer.experiment(model, criteria, optimizer, scheduler, train_loader_aug, valid_loader, 10)

In [14]:
# model, params_to_optimize = models_repo.inception3_extra_layers_test(num_classes, top_layers_to_freeze=15, debug=True)
model, params_to_optimize = models_repo.resnet34_extra_layers(num_classes, top_layers_to_freeze=8, debug=True)

<class 'torch.nn.modules.conv.Conv2d'> 1 0
<class 'torch.nn.modules.batchnorm.BatchNorm2d'> 2 0
<class 'torch.nn.modules.activation.ReLU'> 0 0
<class 'torch.nn.modules.pooling.MaxPool2d'> 0 0
<class 'torch.nn.modules.container.Sequential'> 18 0
<class 'torch.nn.modules.container.Sequential'> 27 0
<class 'torch.nn.modules.container.Sequential'> 39 0
<class 'torch.nn.modules.container.Sequential'> 21 0
<class 'mylib.models_repo.AdaptiveConcatPool2d'> 0 0
<class 'mylib.models_repo.Flatten'> 0 0
<class 'torch.nn.modules.batchnorm.BatchNorm1d'> 2 2
<class 'torch.nn.modules.linear.Linear'> 2 2
<class 'torch.nn.modules.dropout.Dropout'> 0 0
<class 'torch.nn.modules.activation.ReLU'> 0 0
<class 'torch.nn.modules.batchnorm.BatchNorm1d'> 2 2
<class 'torch.nn.modules.linear.Linear'> 2 2
<class 'torch.nn.modules.dropout.Dropout'> 0 0
<class 'torch.nn.modules.activation.ReLU'> 0 0
<class 'torch.nn.modules.batchnorm.BatchNorm1d'> 2 2
<class 'torch.nn.modules.dropout.Dropout'> 0 0
<class 'torch.nn.mo

In [15]:
model, params_to_optimize = models_repo.resnet34_extra_layers(num_classes, top_layers_to_freeze=7)
criteria, optimizer = optimizer_repo.sgd(model, 0.01, params_to_optimize=params_to_optimize)
scheduler = scheduler_repo.step_lr(optimizer, 20, 0.01)

ret = trainer.train(model, criteria, optimizer, scheduler, train_loader_aug, valid_loader, 5)
model, train_loss_trend, val_loss_trend, best_epoch, best_accuracy, best_loss = ret

A Jupyter Widget


Epoch 1/5


60it [00:14,  4.17it/s]
21it [00:04,  4.93it/s]

Training Loss  : 0.032192950901783264 , Acc: 0.16348667015159435
Validation Loss: 0.023313436526732685 , Acc: 0.5062256809338521
############################### Better model found

Epoch 2/5



60it [00:14,  4.25it/s]
21it [00:04,  5.06it/s]

...

Training Loss  : 0.014027793963293225 , Acc: 0.5948771562990068
Validation Loss: 0.01070841557784767 , Acc: 0.6976653696498054
############################### Better model found

Epoch 4/5



60it [00:14,  4.25it/s]
21it [00:04,  5.00it/s]

Training Loss  : 0.011106352748761174 , Acc: 0.6583899634082593
Validation Loss: 0.00898708471528287 , Acc: 0.724124513618677
############################### Better model found

Epoch 5/5



60it [00:14,  4.23it/s]
21it [00:04,  4.95it/s]

Training Loss  : 0.009516675264749362 , Acc: 0.6944589649764767
Validation Loss: 0.008372858178290876 , Acc: 0.7229571984435798



In [ ]:
plt.plot(train_loss_trend)
plt.show()

In [ ]:
plt.plot(val_loss_trend)
plt.show()

In [ ]:
f = model_saver.save_checkpoint(best_epoch, 'inception3', best_model, best_accuracy, 
                                best_loss, optimizer, 'dog.breed')

In [ ]:
best_model.eval()

In [ ]:
from PIL import Image
import torch
import torch.autograd as autograd
import os

In [ ]:
def pred(img_path):
    img = Image.open(img_path).convert('RGB')
    img = trans_valid(img)
    img = autograd.Variable(torch.unsqueeze(img, 0).cuda())
    bm = best_model.cuda()
    output = bm(img)
    arr = output.data.cpu().numpy()[0]
    arr = np.exp(arr) / np.sum(np.exp(arr))
    return arr


In [ ]:
f = open('/tmp/submission.csv', 'w')


for file in os.listdir('/home/as/datasets/kaggle.dog.breed/test'):
    p = os.path.join('/home/as/datasets/kaggle.dog.breed/test', file)
    arr = pred(p)
    s = ''
    for a in arr:
        s += ',' + str(a)
    f.write(file.replace('.jpg', '') + s + '\n')
f.close()

### Experiments from here on

In [ ]:
arr

In [ ]:
np.argmax(arr)
np.argmax(output.data.cpu().numpy()[0])
breeds[41]
output.data.cpu().numpy()[0][59]

In [ ]:
h = []
w = []

for file in os.listdir('/home/as/datasets/kaggle.dog.breed/train'):
    p = os.path.join('/home/as/datasets/kaggle.dog.breed/train', file)
    img = Image.open(p).convert('RGB')
    h.append(img.size[0])
    w.append(img.size[1])



In [ ]:
import numpy as np
print(340 - np.min(h))
print(340 - np.min(w))

In [ ]:
import random

In [ ]:
random.randint(0, -1)

In [ ]:
import torchvision.models as models

def children(m): return m if isinstance(m, (list, tuple)) else list(m.children())

def num_features(m):
    c=children(m)
    if len(c)==0: return None
    for l in reversed(c):
        if hasattr(l, 'num_features'): return l.num_features
        res = num_features(l)
        if res is not None: return res

In [ ]:
model = models.inception_v3(pretrained=True)

In [ ]:
for c in model.children():
    print(type(c))

In [ ]:
model = models.inception_v3(pretrained=True)
modeli = models.inception_v3(pretrained=True)
inception3_layers_to_extract = 17

layers_list = list(model.children())
feature_extracting_layers = layers_list[:inception3_layers_to_extract]

feature_extracting_layers += [models_repo.AdaptiveConcatPool2d(), models_repo.Flatten()]

In [ ]:
model